# 06-06

6000668- Transition matrices + continous
    - Remove Constant Features
    - BoxCox
    - Center and Scaling
    - Euclidean Distance
- Categorical One Hot Encoded Features
    - Jaccard
- Weighted rank result of euclidean and jackard

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyJaccard

from ipynb.fs.full.SearchInfo import searchByID, searchBySpotify, getTrackSpotifyInfo
from ipynb.fs.full.Utils import append_df_to_csv,transformToPercentage

In [2]:
cover = loadCoverGroups()
listCovers = cover[0]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [3]:
cover

[[180849,
  6001027,
  1686718,
  5000013,
  5000001,
  5000007,
  5000008,
  5000023,
  5000017,
  5000019,
  5000020,
  5000018,
  5000011,
  5000012],
 [6001971, 6001984],
 [6002492, 6002491],
 [6000699, 6001165, 6000182],
 [6000104, 6000273],
 [6000768, 6000784],
 [6001931, 6001748],
 [6002043, 6002073],
 [6002244, 6000194],
 [6001209, 6000606],
 [6000352, 6001248],
 [6000549, 6001508, 6001242],
 [6001918, 6001997],
 [6001651, 6001900],
 [6000263, 6001570, 6002374],
 [6001822, 6001653],
 [6000763, 6000668],
 [6000764, 6000667],
 [6001543, 6001541],
 [6000762, 6000773],
 [6000771, 6000670],
 [6001747, 6001952],
 [6000548, 6002203],
 [6000172, 6002133],
 [6001762, 6001675],
 [6002147, 6002146],
 [6000767, 6000775],
 [6000766, 6000673],
 [6001506, 6000529],
 [6000397, 6000409],
 [6001914, 6002001],
 [6002482, 6002454],
 [6000765, 6000783],
 [6000761, 6000782],
 [6000814, 6000813],
 [6001224, 6000140]]

In [4]:
originalSongId

180849

In [5]:
df = pd.read_csv('./data/unified/05-allDataContinousCategoricalTransition.csv')
df = df.iloc[:, 1:]
encodedFeaturesNames = pd.read_csv('./data/unified/05-encodedFeatureNames.csv')
encodedFeaturesNames = encodedFeaturesNames['0'].values.tolist()
encodedFeaturesNames.append('id')


In [6]:
df.columns

Index(['index', 'id', 'danceability', 'energy', 'speechiness', 'acousticness',
       'liveness', 'valence', 'tempo', '0',
       ...
       'mood_5', 'mood_6', 'mood_7', 'mood_8', 'mood_9', 'mood_10', 'mood_11',
       'instrumentalness_cat_0', 'instrumentalness_cat_1',
       'instrumentalness_cat_2'],
      dtype='object', length=1076)

In [7]:
dfE = df[encodedFeaturesNames]
dfE = dfE.sort_values('id')
ids = dfE.id
dfE.drop(['id'], axis=1, inplace=True)
dfE.reset_index(drop=True, inplace=True)

In [8]:
dfE = removeConstantFeatures(dfE)

In [9]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfE)
coverIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfE)

In [10]:
##[0,1,1], [1,0,1] -> 0.66 Diff -> 0.33 Similarity
ranksJaccard = applyJaccard(dfE, ids, originalSongVector)

In [11]:
ranksJaccard

,id,rank
0,106522,0.0
1,1029726,0.0
2,3653915,0.0
3,6001258,0.0
4,466649,0.0
...,...,...
5972,6000126,1.0
5973,783974,1.0
5974,6000825,1.0
5975,6001510,1.0


In [12]:
ranksJaccard['rank'].value_counts().sort_index()

0.000000      10
0.333333     216
0.571429    1413
0.750000    3023
0.888889    1237
1.000000      78
Name: rank, dtype: int64

## Continous features

In [13]:
encodedFeaturesNames.remove('id')
df = df[df.columns.difference(encodedFeaturesNames)]
df = df.sort_values('id')
ids = df.id
df.drop(['id'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

## Pipeline

In [14]:
df = removeConstantFeatures(df)
df = centerAndScale(df)

mean:  [ 0. -0. -0. ... -0.  0.  0.]
std:  [1. 1. 1. ... 1. 1. 1.]


### Original Song & Cover Song

In [15]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, df)
coverIndex,coverSongVector = getIndexAndVector(coverSongId, ids, df)

In [16]:
ranksDF  = applyEuclideanDistance(df, ids, originalSongVector)

distance


In [17]:
ranksDF.head(10)

,id,rank
0,180849,0.000000
1,1025154,6.246200
2,569357,6.269484
3,2065851,6.395168
4,1884102,6.478077
5,1210073,6.501521
6,263779,6.518148
7,672021,6.538485
8,1065079,6.544171
9,1048915,6.658681


In [18]:
ranksDF[ranksDF['id']==coverSongId]

,id,rank
69,6001027,7.876343


In [19]:
ranksDF[ranksDF['id'].isin(listCovers)]

,id,rank
0,180849,0.000000
42,5000019,7.569236
69,6001027,7.876343
73,1686718,7.919422
94,5000017,8.061349
133,5000013,8.223692
152,5000018,8.328912
174,5000011,8.411096
218,5000012,8.554556
351,5000001,8.910452


## Weighted rank

In [20]:
ranks = pd.merge(ranksJaccard,ranksDF, on='id')
ranks['new_rank'] = ranks['rank_x'] + ranks['rank_y']
ranks = ranks.sort_values(by='new_rank')
ranks.reset_index(drop=True, inplace=True)


In [21]:
ranks.head(10)

,id,rank_x,rank_y,new_rank
0,180849,0.000000,0.000000,0.000000
1,2065851,0.333333,6.395168,6.728501
2,1025154,0.571429,6.246200,6.817629
3,672021,0.333333,6.538485,6.871819
4,569357,0.750000,6.269484,7.019484
5,1884102,0.571429,6.478077,7.049506
6,3504560,0.333333,6.725566,7.058899
7,1210073,0.571429,6.501521,7.072950
8,1048915,0.571429,6.658681,7.230110
9,263779,0.750000,6.518148,7.268148


In [22]:
ranks[ranks['id']==coverSongId]

,id,rank_x,rank_y,new_rank
19,6001027,0.0,7.876343,7.876343


In [23]:
ranks.loc[ranks.id.isin(listCovers)]

,id,rank_x,rank_y,new_rank
0,180849,0.000000,0.000000,0.000000
19,6001027,0.000000,7.876343,7.876343
21,5000019,0.333333,7.569236,7.902569
62,5000017,0.333333,8.061349,8.394682
71,1686718,0.571429,7.919422,8.490851
108,5000013,0.571429,8.223692,8.795120
130,5000018,0.571429,8.328912,8.900341
196,5000011,0.750000,8.411096,9.161096
220,5000001,0.333333,8.910452,9.243786
244,5000012,0.750000,8.554556,9.304556


In [24]:
ranks.rename(columns = {'new_rank':'rank'}, inplace = True)

In [25]:
# ranks = transformToPercentage(ranks.loc[ranks.id.isin(listCovers)], '0600')
# ranks

,id_chord,id_spotify_track,name,release_date,percentage,rank,position,id_experiment
0,180849,3jButwtJMLx3Ub61BfRiHh,the scientist,2022-06-03,100.000,0.000000,0,0600
1,6001027,75JFxkI2RXiU7L9VXzMkle,NaN,NaN,92.124,7.876343,19,0600
2,5000019,49ilSrZNQcle6PYw8rtvzM,NaN,NaN,92.097,7.902569,21,0600
3,5000017,34dnNAUoIPcwnK0RtVMBWZ,NaN,NaN,91.605,8.394682,62,0600
4,1686718,2LbHYfTYW6TipHInIjqdaQ,the scientist,2015-04-28,91.509,8.490851,71,0600
5,5000013,2LbHYfTYW6TipHInIjqdaQ,NaN,NaN,91.205,8.795120,108,0600
6,5000018,49Nzxf1LskunwmZ6EyWUew,NaN,NaN,91.100,8.900341,130,0600
7,5000011,5FoM2ZWwhgTxHFFh2stUw6,NaN,NaN,90.839,9.161096,196,0600
8,5000001,025vpSSyndhVZezl8rUF6L,NaN,NaN,90.756,9.243786,220,0600
9,5000012,7tn1tcxenMwDXrYTOjS3z1,NaN,NaN,90.695,9.304556,244,0600
